# Bank Marketing Data Set

Im folgenden wird der `Bank Marketing Data Set` aus dem UCI Machine Learning Repository verwendet. Dieser Datensatz enthält Informationen über Kunden, die ein Bankkonto eröffnen möchten. Ziel ist es, ein Modell zu trainieren, das vorhersagen kann, ob ein Kunde ein Konto eröffnen wird oder nicht.

**Matrikel-Nr.**: 1946566

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from pathlib import Path
import json
import requests, zipfile, io

sns.set("notebook", font_scale=1.5, style="white", rc={"figure.figsize":(20, 8)})

In [ ]:
data_path = Path("./")

req = requests.get("https://archive.ics.uci.edu/ml/machine-learning-databases/00222/bank.zip")

zip_file = zipfile.ZipFile(io.BytesIO(req.content))
zip_file.extractall(data_path)

## Data Analysis 1/2

---

Aus den Datensatz gehen folgende Spalten hervor:

**Kundendaten**:

- `age`: Alter des Kunden
- `job`: Beruf des Kunden
- `marital`: Familienstand des Kunden
- `education`: Bildungsstand des Kunden
- `default`: Ob der Kunde ein Kreditkartenkonto hat
- `balance`: Kontostand des Kunden
- `housing`: Ob der Kunde ein Hypothekarkredit hat
- `loan`: Ob der Kunde ein Privatkredit hat

**Letzter Kontakt**:

- `contact`: Art des letzten Kontakts
- `day`: Tag des letzten Kontakts
- `month`: Monat des letzten Kontakts
- `duration`: Dauer des letzten Kontakts in Sekunden

**Andere**:

- `campaign`: Anzahl der Kontakte während dieser Kampagne
- `pdays`: Anzahl der Tage seit dem letzten Kontakt zu einem anderen Kunden
- `previous`: Anzahl der Kontakte zu einem anderen Kunden vor dieser Kampagne
- `poutcome`: Ergebnis der vorherigen Kampagne
- `y`: Ob der Kunde ein Konto eröffnet hat

In [ ]:
df_small = pd.read_csv(data_path / "bank.csv", sep=';')
df_large = pd.read_csv(data_path / "bank-full.csv", sep=';')

In [ ]:
df_large.head()

Aus dem Datensatz geht hervor, dass die Kunden ein **Durschnittsalter** von 41 Jahren haben. Außerdem haben die Kunden Durchschnittlich circa $ `1360` auf dem Konto. 

In [ ]:
df_large.describe()

In [ ]:
df_large.info(memory_usage='deep')

In [ ]:
# Check for duplicates
df_large.duplicated().value_counts()

In [ ]:
# Check for missing values
any(df_large.isna().sum())

In [ ]:
category_col = [col for col in df_large.columns if df_large[col].nunique() <= 15]
category_col_values = {
    col: df_large[col].value_counts().to_dict() 
    for col in df_large.columns if df_large[col].nunique() <= 15
}

with open(data_path / "category_col_values.json", "w") as f:
    json.dump(category_col_values, f)
    
print(category_col)

## Data Preparation

---

#### Cleaning

In [ ]:
def prepare_pipeline(df: pd.DataFrame) -> pd.DataFrame:
    df_types = {
        "age": np.int8,
        "job": "category",
        "marital": "category",
        "education": "category",
        "default": "category",
        "balance": np.int32,
        "housing": "category",
        "loan": "category",
        "contact": "category",
        "day": np.int8,
        "month": "category",
        "duration": np.int16,
        "campaign": np.int8,
        "pdays": np.int16,
        "previous": np.int8,
        "poutcome": "category",
        "y": "category"
    }
    
    df = df.astype(df_types)
    
    return df

In [ ]:
df_prep = prepare_pipeline(df_large.copy())

In [ ]:
df_prep.info(memory_usage='deep')

#### Encoding



In [ ]:
def encode_pipeline(df: pd.DataFrame) -> pd.DataFrame:
    df["default"] = df["default"].map({"no": 0, "yes": 1}).astype(np.int8)
    df["housing"] = df["housing"].map({"no": 0, "yes": 1}).astype(np.int8)
    df["loan"] = df["loan"].map({"no": 0, "yes": 1}).astype(np.int8)
    df["month"] = df["month"].map({
        "jan": 1, "feb": 2, "mar": 3, "apr": 4, "may": 5, "jun": 6,
        "jul": 7, "aug": 8, "sep": 9, "oct": 10, "nov": 11, "dec": 12
    }).astype(np.int8)
    df["y"] = df["y"].map({"no": 0, "yes": 1}).astype(np.int8)
    
    df = pd.get_dummies(df, columns=["job", "education", "poutcome", "marital", "contact"])
    
    return df

In [ ]:
df_enc = encode_pipeline(df_prep.copy())

In [ ]:
df_enc.describe()

In [ ]:
df_enc.info(memory_usage='deep')

## Data Analysis 2/2

### Plotting

#### Subscribers - Term Deposit

Aus der Grafik wir ersichtlich, dass es deutlich mehr Kunden gibt, die kein Konto bei der Bank besitzen.

In [ ]:
sns.histplot(df_large, x="y", discrete=True)

#### Properties

Die Menge der Kunden in dem Alter von `25 - 60` Jahren ist sehr hoch. Dies könnte daran liegen, dass dies die Altersgruppe ist, in der die meisten Menschen arbeiten und somit ein Konto bei der Bank benötigen.

Der Job, Ehestatus und der Bildung variiert sehr stark von Kategorie zu Kategorie. Die Trends, ob ein Kunde ein Konto besitzt oder nicht, sind unabhängig von diesen Eigenschaften sehr ähnlich.

Die meisten Kunden sind im Bereich *Management*, *Blue Collar* und *Techniker* tätig.

Die meisten Kunden sind außerdem verheiratet und haben einen höheren Bildungsabschluss.

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(25, 25))

sns.histplot(df_large, x="age", ax=axs[0][0])
sns.countplot(x="job", hue="y", data=df_large, ax=axs[0][1])
sns.countplot(x="marital", hue="y", data=df_large, ax=axs[1][0])
sns.countplot(x="education", hue="y", data=df_large, ax=axs[1][1])

axs[0][0].set_title("Age")
axs[0][1].set_title("Job")
axs[1][0].set_title("Marital")
axs[1][1].set_title("Education")

axs[0][1].tick_params(axis='x', rotation=35)

In [ ]:
sns.boxenplot(df_large, x="job", y="age", hue="y")

plt.title("Age by Job")
plt.xticks(rotation=35)

#### Loans and Credits

Wenn ein Kunde ein `default_credit` haben, gibt es keinen Kunden der ebenfalls ein `term_deposit` hat.

Wenn ein Kunde einen Kredit auf sein Haus aufgenommen hat, ist das Verhältnis von Kunden mit einem `term_deposit` leicht höher. Außerdem lässt sich feststellen, dass es mehr Kunden mit einen Kredit auf ihrem Haus gibt, als ohne.

Es gibt deutlich mehr Kunden ohne einen `personal_loan` als mit einem.

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(25, 25))

sns.countplot(x="default", hue="y", data=df_large, ax=axs[0][0])
sns.countplot(x="housing", hue="y", data=df_large, ax=axs[0][1])
sns.countplot(x="loan", hue="y", data=df_large, ax=axs[1][0])

#### Other

In [ ]:
sns.scatterplot(x="duration", y="balance", hue="marital", data=df_large)

### Correlation

Die Korrelation zwischen der Dauer des Gesprächs und ob ein Kunde ein Konto eröffnet korrelieren zu stark miteinander. Dies könnte daran liegen, dass die Kunden, die ein Konto eröffnen, länger mit dem Bankberater sprechen, um mehr Informationen zu erhalten. 

**Columns to Drop**:

- `duration`

In [ ]:
corr_matrix = df_enc.iloc[:, 0:13].corr()
mask = np.zeros_like(corr_matrix)
mask[np.triu_indices_from(mask)] = True

plt.figure(figsize=(20, 20))
sns.heatmap(corr_matrix, mask=mask, square=True, cmap="coolwarm")

In [ ]:
collected_corr = []

for col in corr_matrix.columns:
    for row in corr_matrix.index:
        if row != col:
            collected_corr.append((col, row, corr_matrix.loc[row, col]))
            
df_collected_corr = pd.DataFrame(collected_corr, columns=["col1", "col2", "corr"])
df_collected_corr["abs_corr"] = df_collected_corr["corr"].abs()
df_collected_corr.sort_values("abs_corr", ascending=False, inplace=True)

top_collected = df_collected_corr[:20]
bayes_filter_1 = top_collected["col1"].value_counts()
bayes_filter_2 = top_collected["col2"].value_counts()

bayes_filter = pd.concat([bayes_filter_1, bayes_filter_2], axis=1).fillna(0).sum(axis=1)

In [ ]:
y_corr = pd.DataFrame(corr_matrix["y"])
y_corr["corr_abs"] = y_corr["y"].abs()
y_corr = y_corr.sort_values(by="corr_abs", ascending=False)
y_corr[1:11]

In [ ]:
filter_naive_bias = y_corr[1:11].index

## Modeling and Evaluation

---

Um die Trainingszeit zu verkürzen, wird der kleinere Datensatz (`bank.csv`) für das Training und Modelling verwendet.

In [ ]:
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.preprocessing import StandardScaler

# TODO: Add more columns to drop
drop_columns = ["y", "pdays"]

In [ ]:
df_prep_model = prepare_pipeline(df_small.copy())
df_enc_model = encode_pipeline(df_prep_model.copy())

In [ ]:
any(df_enc_model.isna().sum())

In [ ]:
X_train = df_enc_model.drop(drop_columns, axis=1)
X_train_scaled = StandardScaler().fit_transform(X_train[["age", "balance", "day", "campaign", "pdays", "previous"]])

Y_train = df_enc_model["y"]

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X_train, Y_train, test_size=0.2, random_state=42)

x_train_scaled, x_test_scaled, y_train, y_test = train_test_split(X_train_scaled, Y_train, test_size=0.2, random_state=42)

In [ ]:
df_y_count = pd.DataFrame([y_train.value_counts(), y_test.value_counts()], index=["train", "test"])
df_y_count["ratio"] = df_y_count[1] / df_y_count[0]

df_y_count

### Modeling

In [ ]:
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import LinearSVC
# Lazy Predict is a handy tool to test multiple models
from lazypredict.Supervised import LazyClassifier

metrics = ['accuracy', 'precision', 'recall', 'f1', 'roc_auc']
collected_results = {}

In [ ]:
lazy_clf = LazyClassifier(verbose=0, ignore_warnings=True, custom_metric=None)

In [ ]:
lazy_clf_results, _ = lazy_clf.fit(x_train, x_test, y_train, y_test)

In [ ]:
lazy_clf_results.sort_values(by="Accuracy", ascending=False)

#### Ada Boost

In [ ]:
ada_clf = AdaBoostClassifier()

In [ ]:
ada_cv_scores = cross_validate(ada_clf, X_train, Y_train, cv=5, scoring=metrics, return_train_score=True)

collected_results["AdaBoost"] = {key: value.mean() for key, value in ada_cv_scores.items() if key.startswith("test_")}

In [ ]:
ada_cv_scores = cross_validate(ada_clf, X_train_scaled, Y_train, cv=5, scoring=metrics, return_train_score=True)

collected_results["AdaBoost_scaled"] = {key: value.mean() for key, value in ada_cv_scores.items() if key.startswith("test_")}

In [ ]:
ConfusionMatrixDisplay(confusion_matrix(y_test, ada_clf.fit(x_train, y_train).predict(x_test)), display_labels=["no", "yes"]).plot()

#### Logistic Regression

In [ ]:
log_clf = GradientBoostingClassifier()

In [ ]:
log_cv_scores = cross_validate(log_clf, X_train, Y_train, cv=5, scoring=metrics, return_train_score=True)

collected_results["GradientBoosting"] = {key: value.mean() for key, value in log_cv_scores.items() if key.startswith("test_")}

In [ ]:
log_cv_scores = cross_validate(log_clf, X_train_scaled, Y_train, cv=5, scoring=metrics, return_train_score=True)

collected_results["GradientBoosting_scaled"] = {key: value.mean() for key, value in log_cv_scores.items() if key.startswith("test_")}

In [ ]:
ConfusionMatrixDisplay(confusion_matrix(y_test, log_clf.fit(x_train, y_train).predict(x_test)), display_labels=["no", "yes"]).plot()

#### Random Forest

In [ ]:
rand_clf = RandomForestClassifier()

In [ ]:
rand_cv_scores = cross_validate(rand_clf, X_train, Y_train, cv=5, scoring=metrics, return_train_score=True)

collected_results["RandomForest"] = {key: value.mean() for key, value in rand_cv_scores.items() if key.startswith("test_")}

In [ ]:
rand_cv_scores = cross_validate(rand_clf, X_train_scaled, Y_train, cv=5, scoring=metrics, return_train_score=True)

collected_results["RandomForest_scaled"] = {key: value.mean() for key, value in rand_cv_scores.items() if key.startswith("test_")}

In [ ]:
ConfusionMatrixDisplay(confusion_matrix(y_test, rand_clf.fit(x_train, y_train).predict(x_test)), display_labels=["no", "yes"]).plot()

#### Gradient Boosting

In [ ]:
gradient_clf = GradientBoostingClassifier()

In [ ]:
gradient_cv_scores = cross_validate(gradient_clf, X_train, Y_train, cv=5, scoring=metrics, return_train_score=True)

collected_results["GradientBoosting"] = {key: value.mean() for key, value in gradient_cv_scores.items() if key.startswith("test_")}

In [ ]:
gradient_cv_scores = cross_validate(gradient_clf, X_train_scaled, Y_train, cv=5, scoring=metrics, return_train_score=True)

collected_results["GradientBoosting_scaled"] = {key: value.mean() for key, value in gradient_cv_scores.items() if key.startswith("test_")}

In [ ]:
ConfusionMatrixDisplay(confusion_matrix(y_test, rand_clf.fit(x_train, y_train).predict(x_test)), display_labels=["no", "yes"]).plot()

#### Support Vector Machine

In [ ]:
svm_clf = LinearSVC()

In [ ]:
svm_cv_scores = cross_validate(svm_clf, X_train, Y_train, cv=5, scoring=metrics, return_train_score=True)

collected_results["SVM"] = {key: value.mean() for key, value in svm_cv_scores.items() if key.startswith("test_")}

In [ ]:
svm_cv_scores = cross_validate(svm_clf, X_train_scaled, Y_train, cv=5, scoring=metrics, return_train_score=True)

collected_results["SVM_scaled"] = {key: value.mean() for key, value in svm_cv_scores.items() if key.startswith("test_")}

In [ ]:
ConfusionMatrixDisplay(confusion_matrix(y_test, svm_clf.fit(x_train, y_train).predict(x_test)), display_labels=["no", "yes"]).plot()

#### Gaussian Naive Bayes

Zuerst wird Naive Bayes auf dem gesamten Datensatz trainiert und anschließend auf dem bereinigten Datensatz. Für den bereinigten Datensatz werden die Spalten mit vielen und hohen Korrelationen entfernt.

In [ ]:
gaussian_clf = GaussianNB()

In [ ]:
X_train_bayes = X_train.drop(filter_naive_bias.drop("duration"), axis=1)

In [ ]:
gaussian_cv_scores = cross_validate(gaussian_clf, X_train, Y_train, cv=5, scoring=metrics, return_train_score=True)

collected_results["GaussianNB"] = {key: value.mean() for key, value in gaussian_cv_scores.items() if key.startswith("test_")}

In [ ]:
gussian_cv_scores = cross_validate(gaussian_clf, X_train_scaled, Y_train, cv=5, scoring=metrics, return_train_score=True)

collected_results["GaussianNB_scaled"] = {key: value.mean() for key, value in gussian_cv_scores.items() if key.startswith("test_")}

In [ ]:
gaussian_cv_scores = cross_validate(gaussian_clf, X_train_bayes, Y_train, cv=5, scoring=metrics, return_train_score=True)

collected_results["GaussianNB_cleaned"] = {key: value.mean() for key, value in gaussian_cv_scores.items() if key.startswith("test_")}

In [ ]:
ConfusionMatrixDisplay(confusion_matrix(y_test, gaussian_clf.fit(x_train, y_train).predict(x_test)), display_labels=["no", "yes"]).plot()

#### Evaluation

In [ ]:
renamed_columns = {
    "test_accuracy": "Accuracy",
    "test_precision": "Precision",
    "test_recall": "Recall",
    "test_f1": "F1",
    "test_roc_auc": "ROC AUC"
}

In [ ]:
important_random = pd.DataFrame(rand_clf.fit(X_train, Y_train).feature_importances_, index=X_train.columns, columns=["Importance"]).sort_values("Importance", ascending=False)

important_random.sort_values("Importance", ascending=False)[:10].plot(kind="barh")

In [ ]:
df_results = pd.DataFrame.from_dict(collected_results, orient='index').rename(columns=renamed_columns)
df_results.sort_values(by="Accuracy", ascending=False)

### Hyperparameter Tuning

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
grid_search = GridSearchCV(
    rand_clf,
    param_grid={
        "n_estimators": [50, 100, 200],
        "max_depth": [2, 3, 5, 7],
        "min_samples_split": [2, 3, 5, 10],
        "min_samples_leaf": [1, 2, 3, 5],
    },
    cv=5,
    scoring="accuracy",
    n_jobs=-1,
    verbose=1
)

grid_search.fit(X_train_scaled, Y_train)